# Unit Testing ML Code: Hands-on Exercise (Model Quality)

## In this notebook we will explore unit tests for *model prediction quality*

#### We will use a classic toy dataset: the Iris plants dataset, which comes included with scikit-learn
Dataset details: https://scikit-learn.org/stable/datasets/index.html#iris-plants-dataset

As we progress through the course, the complexity of examples will increase, but we will start with something basic. This notebook is designed so that it can be run in isolation, once the setup steps described below are complete. Cells should be run one after the other without skipping any.

### Setup

Let's begin by importing the dataset and the libraries we are going to use. Make sure you have run `pip install -r requirements.txt` on requirements file located in the same directory as this notebook. We recommend doing this in a separate virtual environment (see dedicated setup lecture).

If you need a refresher on jupyter, pandas or numpy, there are some links to resources in the section notes.

In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np

# Access the iris dataset from sklearn
iris = datasets.load_iris()

# Load the iris data into a pandas dataframe. The `data` and `feature_names`
# attributes of the dataset are added by default by sklearn. We use them to
# specify the columns of our dataframes.
iris_frame = pd.DataFrame(iris.data, columns=iris.feature_names)

# Create a "target" column in our dataframe, and set the values to the correct
# classifications from the dataset.
iris_frame['target'] = iris.target

### Create the Pipelines

Below we use both pipelines from the previous exercises:

- `SimplePipeline` from the testing inputs lecture
- `PipelineWithFeatureEngineering` from the testing data engineering lecture

The pipelines have not been changed. We use both so that we can compare predictions between them in our tests.

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


class SimplePipeline:
    def __init__(self):
        self.frame = None
        # Shorthand to specify that each value should start out as
        # None when the class is instantiated.
        self.X_train, self.X_test, self.y_train, self.Y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        """Load the dataset and perform train test split."""
        # fetch from sklearn
        dataset = datasets.load_iris()
        
        # remove units ' (cm)' from variable names
        self.feature_names = [fn[:-5] for fn in dataset.feature_names]
        self.frame = pd.DataFrame(dataset.data, columns=self.feature_names)
        self.frame['target'] = dataset.target
        
        # we divide the data set using the train_test_split function from sklearn, 
        # which takes as parameters, the dataframe with the predictor variables, 
        # then the target, then the percentage of data to assign to the test set, 
        # and finally the random_state to ensure reproducibility.
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[self.feature_names], self.frame.target, test_size=0.65, random_state=42)
        
    def train(self, algorithm=LogisticRegression):
        
        # we set up a LogisticRegression classifier with default parameters
        self.model = algorithm(solver='lbfgs', multi_class='auto')
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        return self.model.predict(input_data)
        
    def get_accuracy(self):
        
        # use our X_test and y_test values generated when we used
        # `train_test_split` to test accuracy.
        # score is a method on the Logisitic Regression that 
        # returns the accuracy by default, but can be changed to other metrics, see: 
        # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        """Helper method to run multiple pipeline methods with one call."""
        self.load_dataset()
        self.train()

In [3]:
from sklearn.preprocessing import StandardScaler


class PipelineWithDataEngineering(SimplePipeline):
    def __init__(self):
        # Call the inherited SimplePipeline __init__ method first.
        super().__init__()
        
        # scaler to standardize the variables in the dataset
        self.scaler = StandardScaler()
        # Train the scaler once upon pipeline instantiation:
        # Compute the mean and standard deviation based on the training data
        self.scaler.fit(self.X_train)
    
    def apply_scaler(self):
        # Scale the test and training data to be of mean 0 and of unit variance
        self.X_train = self.scaler.transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)
        
    def predict(self, input_data):
        # apply scaler transform on inputs before predictions
        scaled_input_data = self.scaler.transform(input_data)
        return self.model.predict(scaled_input_data)
                  
    def run_pipeline(self):
        """Helper method to run multiple pipeline methods with one call."""
        self.load_dataset()
        self.apply_scaler()  # updated in the this class
        self.train()

### Now we Unit Test

We will employ a few different tests for model prediction quality:

1. A benchmark test: checking model accuracy against a simple benchmark
2. A differential test: checking model accuracy from one version to the next

To begin, let's establish a base line. The simplest baseline is predicting the most common class. If we run: 

In [4]:
iris_frame['target'].value_counts()

2    50
1    50
0    50
Name: target, dtype: int64

We can see that there an equal number of classifications for the 3 flower types. Let's check the accuracy when always predicting classification 1. Obviously this is a very low benchmark (circa 33% accuracy on the dataset), but it serves to illustrate the sort of checks you should be running with your models. If this test fails, then our model accuracy is terrible and we have probably introduced a severe bug into our code.

In [5]:
import unittest
from sklearn.metrics import mean_squared_error, accuracy_score


class TestIrisPredictions(unittest.TestCase):
    def setUp(self):
        # We prepare both pipelines for use in the tests
        self.pipeline_v1 = SimplePipeline()
        self.pipeline_v2 = PipelineWithDataEngineering()
        self.pipeline_v1.run_pipeline()
        self.pipeline_v2.run_pipeline()
        
        # the benchmark is simply the same classification value for 
        # for every test entry
        self.benchmark_predictions = [1.0] * len(self.pipeline_v1.y_test)

    def test_accuracy_higher_than_benchmark(self):
        # Given
        benchmark_accuracy = accuracy_score(
            y_true=self.pipeline_v1.y_test,
            y_pred=self.benchmark_predictions)
        
        predictions = self.pipeline_v1.predict(self.pipeline_v1.X_test)
        
        # When
        actual_accuracy = accuracy_score(
            y_true=self.pipeline_v1.y_test,
            y_pred=predictions)
        
        # Then
        print(f'model accuracy: {actual_accuracy}, benchmark accuracy: {benchmark_accuracy}')
        self.assertTrue(actual_accuracy > benchmark_accuracy)
        
    def test_accuracy_compared_to_previous_version(self):
        # When
        v1_accuracy = self.pipeline_v1.get_accuracy()
        v2_accuracy = self.pipeline_v2.get_accuracy()
        print(f'pipeline v1 accuracy: {v1_accuracy}')
        print(f'pipeline v2 accuracy: {v2_accuracy}')
        
        # Then
        self.assertTrue(v2_accuracy >= v1_accuracy)

In [6]:
import sys


suite = unittest.TestLoader().loadTestsFromTestCase(TestIrisPredictions)
unittest.TextTestRunner(verbosity=1, stream=sys.stderr).run(suite)

F.

pipeline v1 accuracy: 0.9693877551020408
pipeline v2 accuracy: 0.9591836734693877
model accuracy: 0.9693877551020408, benchmark accuracy: 0.32653061224489793



FAIL: test_accuracy_compared_to_previous_version (__main__.TestIrisPredictions)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\geoff\AppData\Local\Temp/ipykernel_17944/2220512049.py", line 42, in test_accuracy_compared_to_previous_version
    self.assertTrue(v2_accuracy >= v1_accuracy)
AssertionError: False is not true

----------------------------------------------------------------------
Ran 2 tests in 0.189s

FAILED (failures=1)


<unittest.runner.TextTestResult run=2 errors=0 failures=1>

## Model Quality Testing: Hands-on Exercise
1. Change the SimplePipeline class so that the benchmark test fails. Do you understand why the test is failing?

2. Change either the SimplePipeline or the PipelineWithDataEngineering classes so that `test_accuracy_compared_to_previous_version` **passes**. 

These tests are a little more open ended than others we have looked at, don't worry if you find them tricky!

### 1. Change the SimplePipeline class so that the benchmark test fails

In [7]:
import unittest
from sklearn.metrics import mean_squared_error, accuracy_score


class TestIrisPredictionsGN(unittest.TestCase):
    def setUp(self):
        # We prepare both pipelines for use in the tests
        self.pipeline_v1 = SimplePipeline()
        self.pipeline_v2 = PipelineWithDataEngineering()
        self.pipeline_v1.run_pipeline()
        self.pipeline_v2.run_pipeline()
        
        # the benchmark is simply the same classification value for 
        # for every test entry

        # Set the benchmark equal to the target to fail the test!
        self.benchmark_predictions = self.pipeline_v1.y_test

    def test_accuracy_higher_than_benchmark(self):
        # Given
        benchmark_accuracy = accuracy_score(
            y_true=self.pipeline_v1.y_test,
            y_pred=self.benchmark_predictions)
        
        predictions = self.pipeline_v1.predict(self.pipeline_v1.X_test)
        
        # When
        actual_accuracy = accuracy_score(
            y_true=self.pipeline_v1.y_test,
            y_pred=predictions)
        
        # Then
        print(f'model accuracy: {actual_accuracy}, benchmark accuracy: {benchmark_accuracy}')
        self.assertTrue(actual_accuracy > benchmark_accuracy)
        
    def test_accuracy_compared_to_previous_version(self):
        # When
        v1_accuracy = self.pipeline_v1.get_accuracy()
        v2_accuracy = self.pipeline_v2.get_accuracy()
        print(f'pipeline v1 accuracy: {v1_accuracy}')
        print(f'pipeline v2 accuracy: {v2_accuracy}')
        
        # Then
        self.assertTrue(v2_accuracy >= v1_accuracy)

In [8]:
import sys


suite = unittest.TestLoader().loadTestsFromTestCase(TestIrisPredictionsGN)
unittest.TextTestRunner(verbosity=1, stream=sys.stderr).run(suite)

FF

pipeline v1 accuracy: 0.9693877551020408
pipeline v2 accuracy: 0.9591836734693877
model accuracy: 0.9693877551020408, benchmark accuracy: 1.0



FAIL: test_accuracy_compared_to_previous_version (__main__.TestIrisPredictionsGN)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\geoff\AppData\Local\Temp/ipykernel_17944/119909977.py", line 44, in test_accuracy_compared_to_previous_version
    self.assertTrue(v2_accuracy >= v1_accuracy)
AssertionError: False is not true

FAIL: test_accuracy_higher_than_benchmark (__main__.TestIrisPredictionsGN)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\geoff\AppData\Local\Temp/ipykernel_17944/119909977.py", line 34, in test_accuracy_higher_than_benchmark
    self.assertTrue(actual_accuracy > benchmark_accuracy)
AssertionError: False is not true

----------------------------------------------------------------------
Ran 2 tests in 0.161s

FAILED (failures=2)


<unittest.runner.TextTestResult run=2 errors=0 failures=2>

### 2. Change either the SimplePipeline or the PipelineWithDataEngineering classes so that 'test_accuracy_compared_to_previous_version' passes

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


class SimplePipelineGN:
    def __init__(self):
        self.frame = None
        # Shorthand to specify that each value should start out as
        # None when the class is instantiated.
        self.X_train, self.X_test, self.y_train, self.Y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        """Load the dataset and perform train test split."""
        # fetch from sklearn
        dataset = datasets.load_iris()
        
        # remove units ' (cm)' from variable names
        self.feature_names = [fn[:-5] for fn in dataset.feature_names]
        self.frame = pd.DataFrame(dataset.data, columns=self.feature_names)
        self.frame['target'] = dataset.target
        
        # we divide the data set using the train_test_split function from sklearn, 
        # which takes as parameters, the dataframe with the predictor variables, 
        # then the target, then the percentage of data to assign to the test set, 
        # and finally the random_state to ensure reproducibility.
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[self.feature_names], self.frame.target, test_size=0.65, random_state=42)
        
    def train(self, algorithm=LinearRegression):
        
        # Replace logistic regression with linear regression
        self.model = algorithm()
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        return np.round(self.model.predict(input_data))
        
    def get_accuracy(self):
        
        # use our X_test and y_test values generated when we used
        # `train_test_split` to test accuracy.
        # score is a method on the Logisitic Regression that 
        # returns the accuracy by default, but can be changed to other metrics, see: 
        # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        """Helper method to run multiple pipeline methods with one call."""
        self.load_dataset()
        self.train()


In [19]:
import unittest
from sklearn.metrics import mean_squared_error, accuracy_score


class TestIrisPredictionsGN2(unittest.TestCase):
    def setUp(self):
        # We prepare both pipelines for use in the tests
        self.pipeline_v1 = SimplePipelineGN()
        self.pipeline_v2 = PipelineWithDataEngineering()
        self.pipeline_v1.run_pipeline()
        self.pipeline_v2.run_pipeline()
        
        # the benchmark is simply the same classification value for 
        # for every test entry
        self.benchmark_predictions = [1.0] * len(self.pipeline_v1.y_test)

    def test_accuracy_higher_than_benchmark(self):
        # Given
        benchmark_accuracy = accuracy_score(
            y_true=self.pipeline_v1.y_test,
            y_pred=self.benchmark_predictions)
        
        predictions = self.pipeline_v1.predict(self.pipeline_v1.X_test)
        
        # When
        actual_accuracy = accuracy_score(
            y_true=self.pipeline_v1.y_test,
            y_pred=predictions)
        
        # Then
        print(f'model accuracy: {actual_accuracy}, benchmark accuracy: {benchmark_accuracy}')
        self.assertTrue(actual_accuracy > benchmark_accuracy)
        
    def test_accuracy_compared_to_previous_version(self):
        # When
        v1_accuracy = self.pipeline_v1.get_accuracy()
        v2_accuracy = self.pipeline_v2.get_accuracy()
        print(f'pipeline v1 accuracy: {v1_accuracy}')
        print(f'pipeline v2 accuracy: {v2_accuracy}')
        
        # Then
        self.assertTrue(v2_accuracy >= v1_accuracy)

In [20]:
import sys


suite = unittest.TestLoader().loadTestsFromTestCase(TestIrisPredictionsGN2)
unittest.TextTestRunner(verbosity=1, stream=sys.stderr).run(suite)

..

pipeline v1 accuracy: 0.9272407796710059
pipeline v2 accuracy: 0.9591836734693877
model accuracy: 0.9693877551020408, benchmark accuracy: 0.32653061224489793



----------------------------------------------------------------------
Ran 2 tests in 0.141s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>